<a href="https://colab.research.google.com/github/alibakh62/DeepFake-Kaggle/blob/master/notebook/colab/DeepFakeDetection_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Version 1

Main steps:



*   Videos -> frames (independent images)
*   Pre-trained convnets -> training only the top layer
*   Prediction on video -> Rolling moving average



# Config and Imports

In [0]:
# from google.colab import drive
# drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [1]:
!pip install ipython-autotime

  Created wheel for ipython-autotime: filename=ipython_autotime-0.1-cp36-none-any.whl size=1832 sha256=8b1b40d717bf62141bb40b13d7db9207f916f6984cb3de106d9fd8d8121f8aaa
  Stored in directory: /root/.cache/pip/wheels/d2/df/81/2db1e54bc91002cec40334629bc39cfa86dff540b304ebcd6e
Successfully built ipython-autotime


In [2]:
%load_ext autotime
%tensorflow_version 2.x
%matplotlib inline 

TensorFlow 2.x selected.


In [13]:
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Input, BatchNormalization, Activation
from tensorflow.keras.layers import Conv2D, SeparableConv2D, MaxPooling2D, GlobalAveragePooling2D, GlobalMaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#from tensorflow.keras.applications.imagenet_utils import _obtain_input_shape
from tensorflow.keras.utils import get_file

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler
# from tensorflow.keras.utils import np_utils
from tensorflow.keras import regularizers, optimizers
from tensorflow.keras.optimizers import SGD
import os
import shutil
from glob import glob
import math
import h5py
import matplotlib
import matplotlib.pyplot as plt
import random
#import cv2
from google.colab.patches import cv2_imshow
from sklearn.preprocessing import LabelEncoder
from skimage import io, transform
from sklearn.metrics import accuracy_score
from scipy import misc
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook
import cv2 as cv

time: 26.9 ms


# Models

### Importing pretrained convnets

In [22]:
from tensorflow.keras.applications import Xception

conv_base = Xception(include_top=False)
# conv_base.summary()

time: 2.38 s


In [23]:
# freezing pretrained model
conv_base.trainable = False

# adding top layers
model = Sequential()
model.add(conv_base)
model.add(GlobalAveragePooling2D())
model.add(Dense(1024, activation="selu"))
model.add(Dense(265, activation="selu"))
model.add(Dense(1, activation="sigmoid"))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Model)             (None, None, None, 2048)  20861480  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 2048)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 1024)              2098176   
_________________________________________________________________
dense_4 (Dense)              (None, 265)               271625    
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 266       
Total params: 23,231,547
Trainable params: 2,370,067
Non-trainable params: 20,861,480
_________________________________________________________________
time: 1.18 s


# DataPrep

### Config
***

In [9]:
# configs
MODELS_FOLDER = '/content/drive/My Drive/DeepFake/models'
BASE_FOLDER = '/content/drive/My Drive/DeepFake'
TRAIN_SAMPLE_VIDEOS = 'train_sample_videos'
TEST_VIDEOS = 'test_videos'
TRAIN_FRAMES_FOLDER = 'train_frames'
VALID_FRAMES_FOLDER = 'valid_frames'
TEST_FRAMES_FOLDER = 'test_frames'
DATA_FOLDER = 'data'
TRAIN_FOLDER = 'train'
VALID_FOLDER = 'valid'
TEST_FOLDER = 'test'

time: 6.36 ms


### Meta data
***

In [10]:
def get_meta_from_json(path):
    df = pd.read_json(os.path.join(BASE_FOLDER, path, json_file))
    df = df.T
    return df

train_list = list(os.listdir(os.path.join(BASE_FOLDER, TRAIN_SAMPLE_VIDEOS)))
test_list = list(os.listdir(os.path.join(BASE_FOLDER, TEST_VIDEOS)))
json_file = [file for file in train_list if  file.endswith('json')][0]
meta_train_df = get_meta_from_json(TRAIN_SAMPLE_VIDEOS)
meta_train_df.head()

,label,split,original
aagfhgtpmv.mp4,FAKE,train,vudstovrck.mp4
aapnvogymq.mp4,FAKE,train,jdubbvfswz.mp4
abarnvbtwb.mp4,REAL,train,None
abofeumbvv.mp4,FAKE,train,atvmxvwyns.mp4
abqwwspghj.mp4,FAKE,train,qzimuostzz.mp4


time: 10.9 s


### Converting videos into images
***

In [0]:
#@title Sample or All
sample = False #@param {type:"boolean"}
no_of_samples = 5 #@param {type:"integer"}

def get_frames(cap, name, dirname, resize=None):
    frameRate = cap.get(5) #frame rate
    count = 0
    while(cap.isOpened()):
        frameId = cap.get(1) #current frame number
        ret, frame = cap.read()
        if (ret != True):
            break
        if (frameId % math.floor(frameRate) == 0):
            # storing the frames in a different folder
            if resize is not None:
                # image = np.array(tf.image.resize(frame, resize))
                image = cv.resize(frame, resize)
            else:
                image = frame
            if not os.path.isdir(os.path.join(BASE_FOLDER, dirname)):
                os.mkdir(os.path.join(BASE_FOLDER, dirname))
            count += 1
            fn = os.path.splitext(name)[0] + f"_frame{count}.jpg"
            file_path = os.path.join(BASE_FOLDER, dirname, fn)
            cv.imwrite(file_path, image)
    cap.release()

cnt_max = 2e10
if sample:
    cnt_max = no_of_samples

def walkdir(dirpath):
    for root, dirs, files in os.walk(dirpath):
        for name in files:
            if name != 'metadata.json':
                yield os.path.abspath(os.path.join(dirpath, name)), name

filescount = meta_train_df.index.nunique()

cnt = 1
X_train_full, y_train_full = [], []
for path, name in tqdm_notebook(walkdir(os.path.join(BASE_FOLDER, TRAIN_SAMPLE_VIDEOS)), total=filescount):
    if cnt <= cnt_max:
        cap = cv.VideoCapture(path)
        get_frames(cap, name, TRAIN_FRAMES_FOLDER)
        cnt += 1


time: 42min 54s


#### Getting train data & label
***

In [0]:
from tensorflow.keras.preprocessing import image
from sklearn.model_selection import train_test_split
from sklearn.utils.validation import column_or_1d

images = glob(os.path.join(BASE_FOLDER, f"{TRAIN_FRAMES_FOLDER}/*.jpg"))
X_full, y_full = [], []
for i in tqdm_notebook(images):
    frame_name = i.split('/')[-1]
    image_name = frame_name.split("_")[0]
    label = 1
    if meta_train_df.loc[meta_train_df.index == image_name+'.mp4', "label"].values[0] == 'REAL':
        label = 0
    # reading frames
    img = image.load_img(i, target_size=(512, 512, 3))
    img = image.img_to_array(img)
    img = img/255
    X_full.append(img)
    y_full.append(label)

X = np.array(X_full)
y = column_or_1d(y_full)
print(X.shape)
print(y.shape)

In [0]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state=42, test_size=0.2, stratify=y)
print(f"X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"X_valid: {X_valid.shape}, y_valid: {y_valid.shape}")

NameError: ignored

### Converting videos to images (Keras)
***

#### Creating directories

In [20]:
# base folder
if not os.path.isdir(BASE_FOLDER):
    os.mkdir(BASE_FOLDER)

# data folder
data_dir = os.path.join(BASE_FOLDER, DATA_FOLDER)
if not os.path.isdir(data_dir):
    os.mkdir(data_dir)

# train folder
train_dir = os.path.join(data_dir, TRAIN_FOLDER)
if not os.path.isdir(train_dir):
    os.mkdir(train_dir)

if not os.path.isdir(os.path.join(train_dir, 'REAL')):
    os.mkdir(os.path.join(train_dir, 'REAL'))

if not os.path.isdir(os.path.join(train_dir, 'FAKE')):
    os.mkdir(os.path.join(train_dir, 'FAKE'))

# valid folder
valid_dir = os.path.join(data_dir, VALID_FOLDER)
if not os.path.isdir(valid_dir):
    os.mkdir(valid_dir)

if not os.path.isdir(os.path.join(valid_dir, 'REAL')):
    os.mkdir(os.path.join(valid_dir, 'REAL'))

if not os.path.isdir(os.path.join(valid_dir, 'FAKE')):
    os.mkdir(os.path.join(valid_dir, 'FAKE'))

# test folder
test_dir = os.path.join(data_dir, TEST_FOLDER)
if not os.path.isdir(test_dir):
    os.mkdir(test_dir)



time: 28.6 ms


#### Adding frames to train & validation folders

In [22]:
#@title Sample or All
sample = False #@param {type:"boolean"}
no_of_samples = 2 #@param {type:"integer"}

time: 1.2 ms


In [23]:
#@title Train/Validation Split
train_val_split = 0.7 #@param {type: "number"}

time: 1.2 ms


In [24]:
#TODO: resize not necessary here, will be done in ImageDataGenerator
def get_frames(cap, name, dirname, resize=None):
    frameRate = cap.get(5) #frame rate
    label = meta_train_df.loc[meta_train_df.index == name, "label"].values[0]
    count = 0
    while(cap.isOpened()):
        frameId = cap.get(1) #current frame number
        ret, frame = cap.read()
        if (ret != True):
            break
        if (frameId % math.floor(frameRate) == 0):
            # storing the frames in a different folder
            if resize is not None:
                # image = np.array(tf.image.resize(frame, resize))
                image = cv.resize(frame, resize)
            else:
                image = frame
            count += 1
            fn = os.path.splitext(name)[0] + f"_frame{count}.jpg"
            file_path = os.path.join(BASE_FOLDER, dirname, label, fn)
            cv.imwrite(file_path, image)
    cap.release()

def walkdir(dirpath):
    for root, dirs, files in os.walk(dirpath):
        for name in files:
            if name != 'metadata.json':
                yield os.path.abspath(os.path.join(dirpath, name)), name

cnt_max = 2e10
if sample:
    cnt_max = no_of_samples

filescount = meta_train_df.index.nunique()
traincount = int(filescount * train_val_split)

cnt = 1
for path, name in tqdm_notebook(walkdir(os.path.join(BASE_FOLDER, TRAIN_SAMPLE_VIDEOS)), total=filescount):
    if cnt <= cnt_max:
        cap = cv.VideoCapture(path)
        if cnt <= traincount:
            dirname = os.path.join(DATA_FOLDER, TRAIN_FOLDER)
        else:
            dirname = os.path.join(DATA_FOLDER, VALID_FOLDER)
        get_frames(cap, name, dirname)
        cnt += 1

time: 37min


In [12]:
print(f"Total REAL train: {len(os.listdir(os.path.join(BASE_FOLDER, DATA_FOLDER, TRAIN_FOLDER, 'REAL')))}")
print(f"Total FAKE train: {len(os.listdir(os.path.join(BASE_FOLDER, DATA_FOLDER, TRAIN_FOLDER, 'FAKE')))}")
print(f"Total REAL valid: {len(os.listdir(os.path.join(BASE_FOLDER, DATA_FOLDER, VALID_FOLDER, 'REAL')))}")
print(f"Total FAKE valid: {len(os.listdir(os.path.join(BASE_FOLDER, DATA_FOLDER, VALID_FOLDER, 'FAKE')))}")

Total REAL train: 550
Total FAKE train: 2530
Total REAL valid: 297
Total FAKE valid: 1023
time: 41.8 s


### Image generator for Keras

In [14]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)

train_dir = os.path.join(BASE_FOLDER, DATA_FOLDER, TRAIN_FOLDER)
valid_dir = os.path.join(BASE_FOLDER, DATA_FOLDER, VALID_FOLDER)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(512, 512),
    batch_size=32,
    class_mode='binary')

valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=(512, 512),
    batch_size=32,
    class_mode='binary')

Found 3080 images belonging to 2 classes.
Found 1320 images belonging to 2 classes.
time: 222 ms


In [15]:
for data_batch, label_batch in train_generator:
    print(f"data batch shape: {data_batch.shape}")
    print(f"label batch shape: {label_batch.shape}")
    break

data batch shape: (32, 512, 512, 3)
label batch shape: (32,)
time: 26.8 s


# Training

### Fitting: default

In [0]:
# checkpoints
CHECKPOINT = 'xception_v1.hDF5'
weights_path = os.path.join(MODELS_FOLDER, CHECKPOINT)
from keras.callbacks import ModelCheckpoint
ckp = ModelCheckpoint(weights_path, save_best_only=True, monitor='val_loss', mode='min')

time: 128 ms


Using TensorFlow backend.


In [0]:
# compiling the model
model.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['accuracy'])

time: 71.3 ms


In [0]:
# training the model
model.fit(X_train, y_train, epochs=200, validation_data=(X_valid, y_valid), callbacks=[ckp], batch_size=16)

### Fitting: with generators

In [25]:
from datetime import datetime
timestamp = datetime.now().strftime(format="%Y%m%d_%H%M%S")
CHECKPOINT = f"xception_v1_{timestamp}.h5"
weights_path = os.path.join(MODELS_FOLDER, CHECKPOINT)

from keras.callbacks import ModelCheckpoint
ckp = ModelCheckpoint(weights_path, save_best_only=True, monitor='val_loss', mode='min')

# compiling the model
model.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['accuracy'])

time: 62.9 ms


In [0]:
history = model.fit(
    train_generator,
    steps_per_epoch=100,
    epochs=10,
    validation_data=valid_generator,
    validation_steps=42,
    callbacks=[ckp])

In [0]:
model.save(weights_path)

In [0]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation acc')
plt.title('Training and validation loss')
plt.legend()

plt.show()